## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [10]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = "974386b8-dfe6-43cc-94af-17335974d64a"
resource_group = "mlops-promo"
workspace="mlops-workspace-promo"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [11]:
azureml_tracking_uri = ml_client.workspaces.get(
    ml_client.workspace_name
).mlflow_tracking_uri

In [12]:
import mlflow

mlflow.set_tracking_uri(azureml_tracking_uri)

In [8]:
experiment_name = 'experiment_with_mlflow_first_test'
mlflow.set_experiment(experiment_name)

/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


<Experiment: artifact_location='', creation_time=1730673939921, experiment_id='ffc3247a-0134-4a20-bfa8-643eaad1bbf6', last_update_time=None, lifecycle_stage='active', name='experiment_with_mlflow_first_test', tags={}>

In [13]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='', creation_time=1730673939921, experiment_id='ffc3247a-0134-4a20-bfa8-643eaad1bbf6', last_update_time=None, lifecycle_stage='active', name='experiment_with_mlflow_first_test', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score


with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/11/03 23:47:44 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.17.1, required: mlflow==2.15.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/11/03 23:47:53 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


default artifacts URI: 'azureml://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/974386b8-dfe6-43cc-94af-17335974d64a/resourceGroups/mlops-promo/providers/Microsoft.MachineLearningServices/workspaces/mlops-workspace-promo/experiments/ffc3247a-0134-4a20-bfa8-643eaad1bbf6/runs/7d3b7d20-39a2-4b88-9d59-23877543adf9/artifacts'


2024/11/03 23:47:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run helpful_camel_vt07thxx at: https://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/974386b8-dfe6-43cc-94af-17335974d64a/resourceGroups/mlops-promo/providers/Microsoft.MachineLearningServices/workspaces/mlops-workspace-promo/#/experiments/ffc3247a-0134-4a20-bfa8-643eaad1bbf6/runs/7d3b7d20-39a2-4b88-9d59-23877543adf9.
2024/11/03 23:47:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/974386b8-dfe6-43cc-94af-17335974d64a/resourceGroups/mlops-promo/providers/Microsoft.MachineLearningServices/workspaces/mlops-workspace-promo/#/experiments/ffc3247a-0134-4a20-bfa8-643eaad1bbf6.


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='', creation_time=1730673939921, experiment_id='ffc3247a-0134-4a20-bfa8-643eaad1bbf6', last_update_time=None, lifecycle_stage='active', name='experiment_with_mlflow_first_test', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [10]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1730674069091, description='', last_updated_timestamp=1730674069091, latest_versions=[], name='azureml_7d3b7d20-39a2-4b88-9d59-23877543adf9_output_mlflow_log_model_676068911', tags={}>]

In [14]:
runs = mlflow.search_runs(experiment_names=['experiment_with_mlflow_first_test'])

In [20]:
run_id = runs.iloc[0]['run_id']
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/11/03 23:57:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1730674619300, current_stage='None', description='', last_updated_timestamp=1730674619300, name='iris-classifier', run_id='7d3b7d20-39a2-4b88-9d59-23877543adf9', run_link='', source='azureml://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/974386b8-dfe6-43cc-94af-17335974d64a/resourceGroups/mlops-promo/providers/Microsoft.MachineLearningServices/workspaces/mlops-workspace-promo/experiments/ffc3247a-0134-4a20-bfa8-643eaad1bbf6/runs/7d3b7d20-39a2-4b88-9d59-23877543adf9/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>